# CPU Hotplug

This demonstrates CPU hotplug capabilities of the PM firmware layer which allows CPU cores to be logically brought up or down at runtime.

1. [Introduction](#introduction)
2. [Implementation details](#implementation-details)
    1. [List all CPUs](#list-all-cpus)
    2. [CPU1-offline](#cpu1-offline)
    3. [CPU1-online](#cpu1-online)
2. [CPU Hotplug demo](#cpu-hotplug-demo)
3. [References](#xlnx-pm-wiki)

---

## Introduction <a name="introduction"></a>

The Linux kernel image binary included in Xilinx Petalinux Pre-built BSP for a Versal platform evaluation board is able to dynamically enable and disable CPUs at runtime without having to reboot the system; this is known as CPU "hotplugging".

If you're not using Petalinux pre-built images and compiling using your own config, the kernel option `CONFIG_HOTPLUG_CPU` needs to be enabled while compiling the kernel in order to enable this feature.

This notebook provides a user-friendly and an interactive GUI to demonstrate this feature.

---

## Implementation details <a name="implementation-details"></a>

Before we proceed to the demo, some basics:

##### To list all available CPUs: <a name="list-all-cpus"></a>

In [1]:
cat /proc/cpuinfo

processor	: 0
BogoMIPS	: 200.00
Features	: fp asimd aes pmull sha1 sha2 crc32 cpuid
CPU implementer	: 0x41
CPU architecture: 8
CPU variant	: 0x0
CPU part	: 0xd08
CPU revision	: 3

processor	: 1
BogoMIPS	: 200.00
Features	: fp asimd aes pmull sha1 sha2 crc32 cpuid
CPU implementer	: 0x41
CPU architecture: 8
CPU variant	: 0x0
CPU part	: 0xd08
CPU revision	: 3



We can also use `get_nr_cpus` method from `pmutil` package to obtain the same.

Note that, `pmutil` package provides a few helper methods for demos.

In [2]:
import pmutil
nr_cpus = pmutil.get_nr_cpus()
print("The system has {0} CPU cores.".format(nr_cpus))

The system has 2 CPU cores.


The `debugfs` entries to access CPU cores are listed under `/sys/devices/system/cpu/` path:

In [3]:
ls -lah /sys/devices/system/cpu/

total 0
drwxr-xr-x    7 root     root           0 Mar  7 03:57 ./
drwxr-xr-x    6 root     root           0 Mar  7 03:57 ../
drwxr-xr-x    6 root     root           0 Mar  7 03:57 cpu0/
drwxr-xr-x    6 root     root           0 Mar  7 03:57 cpu1/
drwxr-xr-x    3 root     root           0 Mar  7 04:16 cpufreq/
drwxr-xr-x    2 root     root           0 Mar  7 04:16 hotplug/
-r--r--r--    1 root     root        4.0K Mar  7 04:16 isolated
-r--r--r--    1 root     root        4.0K Mar  7 04:16 kernel_max
-r--r--r--    1 root     root        4.0K Mar  7 04:16 modalias
-r--r--r--    1 root     root        4.0K Mar  7 04:16 offline
-r--r--r--    1 root     root        4.0K Mar  7 03:57 online
-r--r--r--    1 root     root        4.0K Mar  7 04:16 possible
drwxr-xr-x    2 root     root           0 Mar  7 04:16 power/
-r--r--r--    1 root     root        4.0K Mar  7 04:16 present
-rw-r--r--    1 root     root        4.0K Mar  7 04:16 uevent


The files *offline, online, possible, present* represent the CPU masks.
Each CPU folder contains an online file which controls the logical on (1) and off (0) state. Once the CPU is shutdown, it will be removed from `/proc/interrupts`, `/proc/cpuinfo` and should also not be shown visible by the `top` command.

##### To logically shutdown CPU1: <a name="cpu1-offline"></a>

In [4]:
!echo 0 > /sys/devices/system/cpu/cpu1/online
!dmesg | tail -2

[ 2045.689249] CPU1: shutdown
[ 2045.691977] psci: CPU1 killed.


##### To bring CPU1 back online: <a name="cpu1-online"></a>

In [5]:
!echo 1 > /sys/devices/system/cpu/cpu1/online
!dmesg | tail -2

[ 2047.533195] GICv3: CPU1: found redistributor 1 region 0:0x00000000f90a0000
[ 2047.540086] CPU1: Booted secondary processor 0x0000000001 [0x410fd083]


---

## CPU Hotplug Demo <a name="cpu-hotplug-demo"></a>

Since the Versal SoC family has two CPU cores, we have four possible combinations for CPU cores:

| CPU0-state | CPU1-state | Valid/Invalid |
| --- | --- | --- |
| On | On | Valid |
| On | Off | Valid |
| Off | On | Valid |
| Off | Off | Invalid |

Let us run the demo:

In [ ]:
from pmutil import hotplug
hotplug.run_demo()

## References <a name="xlnx-pm-wiki"></a>
[Link to Xilinx PM Wiki Page for Zynq UltraScale＋](https://xilinx-wiki.atlassian.net/wiki/spaces/A/pages/18842232)